## Statistics for collective information cascades in adaptive networks
## Few feautures :
### The mode is a continuous time model
### Dyanmics on the network is based on D-W model and LIF-model. Dyanmics of the network is as follows :
    1. Susceptible (indicated with 1 in this script) can only create (a) or decay (\lambda\) links. 
    2. Infected (indicated with 2 in this script) can only break (b) links but not create/decay any
### No discrete memory term of the D-W model but exponential decay of cumulative dose

### This simulation is for spatial analysis. By default, it starts off with a knn network and is for equal birth and death rates. However, the dynamics on the network does not start until the network topology attains a steady state. This is done in order to make the simulation forget the user bias coming from the initalised knn network

### Note : Changes to be made if the simulation is to be run for unequal birth and death rates
    1. Uncomment the block of code containing "w_minus", the birth and death rates
    2. Comment the function below "equal birth and death rates" in the "Function definitions required for the main part of the script."
    3. Uncomment the function below "unequal birth and death rates" in the "Function definitions required for the main part of the script."

### This simulation outputs the 'out degree' and 'infected fractions' as hdf5 files. Additionally, it can output the in degree and connected components too if required. 

#### For any queries, email rachithaiyappa96@gmail.com
### ----------------------------------------------------------------------------------------------------------------------------------------------------

In [41]:
#importing the required libraries

import networkx as nx #network library to generate inital networks and perform various network operations
import matplotlib.pyplot as plt #for plotting
import numpy as np #for using arrays and vectorising the code wherever possible
import scipy
import random
# from numba import jit #numba precomplier to make the code faster
import pandas as pd
import copy
import warnings
warnings.filterwarnings('ignore')

# Variables used in the simulation

In [42]:
N = 100 #number of nodes in the graph
gamma = 1 #decay of cumulative dose per unit time (second). This is multiplied with dt in the script when required
# gamma = gamma
T = 500 #total number of seconds the simulation runs for 
dt = 0.05 #seconds. Time step of the simulation. Therefore, the number of simulation steps is T/dt''' 

'''number of simulation steps is T/dt'''
'''------------------------------------------------------------------------------------------------------------'''

'''dt_netrun can be thought of as a modified 'dt' which is used to speed up the process of
the network topology reaching steady state. 
For example, supposed the rate at which the network topology changes is very small and maybe almost static,
using dt = 0.05 will take the simulation a long time inorder to stabilise the network 
(for the user intialised network) only after which the dynamics on the network starts.
Thus using dt_netrun = 100 or 10 can result in the network topology taking lesser time to stabilise.
Once the network topology reaches a steady state, the orginal dt = 0.05 is used for the remaining of the simulation
involving the coupling between the dynamics on the network and the dynamics of the network'''

dt_netrun = 10
'''------------------------------------------------------------------------------------------------------------'''

'''Dodds watts parameters'''
p = 1      #rate of dose transfer (per unit time (second))
p = p*dt   #rate of dose transfer per simulation step
r = 1/dt   #rate at which infected recovers if the conditions are met. 
r = r*dt   #This rate is currently set such that an infected ALWAYS recovers if D drops below d*
rho = 1/dt #rate at which recovered individuals become re-susceptible
rho = rho*dt #This rate is currently set such that a recovered ALWAYS immediately becomes re-susceptible. 
#Therefore, for the given values of r and rho, no recovered individuals exist in the system at any point of time
#as infected individuals immediately become resusceptible upon dropping below threshold

d_star = 1            #Individual threshold. Homogeneous
# dose_quantity = 1
D_ini = 3             #The initial cumulative dose of the infected individuals

'''------------------------------------------------------------------------------------------------------------'''

'''initial network parameters'''
# circ_rad = 100     #for a network in which each node is connected to other nodes withing a circle of radius circ_rad
nn = 10            #the number of nearest neighbours(i.e. k) in knn
# average_degree = 6 #for a gnp random network
# total_edges = N*2  #for a gnm random network

'''rewiring parameters'''
rew_r = 25      #radius within which new links can be formed
density = 1/N   #radius within which new links can be formed

'''--------------------------------------------------------------------------------------------------------------'''

'''tolerance to determine initial network topology'''
tol = 0.5
steady_counter_val = 500

### Uncomment the block of code below if unequal rates of birth and death of links are required

In [ ]:
# w_minus = 5 #the death rates 'b' of links (not the same as decay(lambda))
# w_minus = w_minus*dt

***
# Functions for creating different initial networks

### All graphs are directed. This means that the existence of an edge from node A to node B  does not necessarily mean there exists an edge from node B to node A and vice versa.
### The network outputs coordinates, distance matrix, edge_list and adjacency matrix

### The graph should output coordinates, distance matrix, edge_list and adjacency matrix

In [43]:
'''
obtains the number of nodes N
generates N points
calculates euclidean distance between each pair of points
returns the coordinates of the points and the distance matrix which is N*N shaped
with diagonal entries = 0 as this represents each node's distance to itself.
'''
def calc_dist(N) :
    coords = []
    for counter in range(N) :
        coords.append((N*np.random.random(),N*np.random.random()))

    dist_mat = scipy.spatial.distance.cdist(coords,coords)
    return dist_mat,coords

### 1. metric network

In [44]:
'''network = connect to points withing a circle of radius'''
#creates a link between points/nodes which satisfies the conditions of the network
def coupling(dist_mat,circ_rad) :
    edge_list = []
    edge_mask = (dist_mat<circ_rad) & (dist_mat>0)
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(edge_mask))))
    return edge_list

### 2. K nearest neighbour network

In [45]:
'''knn network with N number of nodes and each node connected with nn number of nearest neighbours '''
def knn(dist_mat,nn,N) :
    near_neigh = np.argsort(dist_mat)
    selec_near_neigh = np.zeros((N,nn))
    selec_near_neigh = near_neigh[:,0:nn+1]

    edge_list = []
    for i in range(N) :
        for j in range(1,nn+1) :
            link = [i,selec_near_neigh[i,j]]
            edge_list.append(link)

    return edge_list

### 3. Random Network : GNP type

In [46]:
'''random gnp network with N nodes and having an average degree set by user'''
def rand_network(average_degree,N) :
    z1 = np.random.uniform(size = (N,N))
    E,F = np.meshgrid(np.arange(0,N),np.arange(0,N))
    mask = ((average_degree/N) > z1) & (E!=F)
    adjacency_matrix = np.int64(np.zeros(shape=(N,N)))
    adjacency_matrix[mask] = np.int64(1)
    edge_list = []
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(adjacency_matrix==1))))
    return edge_list,adjacency_matrix

### 4. Random Network : GNM type

In [47]:
'''random gnm network with N nodes and having total number of edges set by the user'''
def rand_net_gnm(total_edges,N) :
    the_graph = nx.gnm_random_graph(N, total_edges,directed=True)
    adjacency_matrix = nx.adjacency_matrix(the_graph)
    adjacency_matrix = np.asarray(adjacency_matrix.todense())
    np.fill_diagonal(adjacency_matrix,0)
    edge_list = []
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(adjacency_matrix==1))))
    return edge_list

***

# Function defintions required for the main part of the script

### Remeber to comment/uncomment the function "rew" based on the requirement of equal/unequal birth and death rates  

In [48]:
# # @jit(nopython=True)
# '''infecting the left part of space.'''
# def left_part_infec(N) :
#     x_coord = []
#     y_coord = []
#     for j in range(len(coords)) :
#         x_coord.append(coords[j][0])
#         y_coord.append(coords[j][1])
#     x_coord = np.asarray(x_coord)
#     y_coord = np.asarray(y_coord)
#     points = np.asarray(np.where(x_coord<(N/5))) 
#     indi_state[points.T] = 2

#     return indi_state

In [49]:
'''infecting 'start' number connected nodes.
This function is obtained from previous works of Dr. Pawel Romanczuk. '''
def InfectNetworkNeighbors(net,seed_node,init_infected_nodes):
     # if in bulk find one node randomly, and infect its neighbours
    infected_nodes = set()
    candidate_nodes = set()
    explored_nodes = set()

    #pick the seed node
    infected_nodes.add(seed_node)
    explored_nodes.add(seed_node)

    curr_node=seed_node

    #add its neighbors to the list of candidates
    for n in net.neighbors(curr_node):
        candidate_nodes.add(int(n))
    #print( curr_node)
    #print( candidate_nodes)

    #while we need to select more nodes...
    while len(infected_nodes) < init_infected_nodes:

        #if there are candidate nodes, select one of them
        if(len(candidate_nodes) > 0):
            new_node = np.random.choice(list(candidate_nodes),1)[0]
            infected_nodes.add(new_node)
            candidate_nodes.remove(new_node)

        elif len(infected_nodes - explored_nodes) > 0:
            curr_node = np.random.choice(list(infected_nodes -
            explored_nodes),1)[0]
            explored_nodes.add(curr_node)
            for n in set(net.neighbors(curr_node)) - infected_nodes:
                candidate_nodes.add(n)

        else:
#             print('Initial node infection step failed')
            return None
    return infected_nodes

In [50]:
''' Function to transer doses. This is first step in the the dynamics on the network.
Dose trasnfer occurs from infected to other infected/susceptible nodes'''
# @jit(nopython=True)
def dose(adj_mat,p,partner_state_prev,d) :

    z1 = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    
    dose_transfer_mask = p > z1
    mod_adj_mat = np.multiply(adj_mat,z1) #modifying adjacency matrix to include the proability of dose transfer
    dose_mask = (p>mod_adj_mat) & (adj_mat!=0) & (partner_state_prev == 2)
    d[dose_mask] = dose_quantity #whenever it is proabable, dose transfer occurs
    
    return d

# ----------------------------------------------------------------------------------------------------
### For equal birth and death rates 

In [1]:
'''Function which controls the coupling between the states of the nodes and the network topology.
This is the dynamics of the network with equal birth and death rates'''


def rew(p_rew,adjacency_matrix,indi_state_for_rew_prev,OD_mesh2,lamb_da,density,rew_r) :
    
    z1_rew_prew = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    z1_rew_lambda = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    
    rew_mask_plus = (p_rew > z1_rew_prew)  & (indi_state_for_rew_prev == 1) & (dist_mat <= rew_r)
    #create links. Dont break any.
    adjacency_matrix[rew_mask_plus] = 1
    rew_mask_minus1 = (p_rew > z1_rew_prew) & (indi_state_for_rew_prev == 2)
    #break links. Dont create any.
    adjacency_matrix[rew_mask_minus1] = 0
    rew_mask_minus2 = ((lamb_da*OD_mesh2) > z1_rew_lambda) & (indi_state_for_rew_prev == 1)
    adjacency_matrix[rew_mask_minus2] = 0
    
    np.fill_diagonal(adjacency_matrix,0)
    
    return adjacency_matrix

### For unequal birth and death rates

In [ ]:
'''Function which controls the coupling between the states of the nodes and the network topology.
This is the dynamics of the network with unequal birth and death rates'''

# def rew(p_rew,adjacency_matrix,indi_state_for_rew_prev,OD_mesh2,lamb_da,density,rew_r) :
    
#     z1_rew_prew = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
#     z1_rew_lambda = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    
#     rew_mask_plus = (p_rew > z1_rew_prew)  & (indi_state_for_rew_prev == 1) & (dist_mat <= rew_r)
#     #create links. Dont break any.
#     adjacency_matrix[rew_mask_plus] = 1
#     rew_mask_minus1 = (w_minus > z1_rew_prew) & (indi_state_for_rew_prev == 2)
#     #break links. Dont create any.
#     adjacency_matrix[rew_mask_minus1] = 0
#     rew_mask_minus2 = ((lamb_da*OD_mesh2) > z1_rew_lambda) & (indi_state_for_rew_prev == 1)
#     adjacency_matrix[rew_mask_minus2] = 0
    
#     np.fill_diagonal(adjacency_matrix,0)
    
#     return adjacency_matrix

# --------------------------------------------------------------------------------------------------

In [52]:
'''Function which allows the network topology to reach steady state before the actual aniamtion/dynamics begin'''

def let_it_run(p_rew_netrun,adjacency_matrix,OD_mesh2,lamb_da_netrun,density,rew_r) :

    z1_rew_prew = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    z1_rew_lambda = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    
    rew_mask_plus = (p_rew_netrun > z1_rew_prew) & (dist_mat <= rew_r)
    #create links. Dont break any.
    adjacency_matrix[rew_mask_plus] = 1
    
    rew_mask_minus2 = ((lamb_da_netrun*OD_mesh2) > z1_rew_lambda)
    adjacency_matrix[rew_mask_minus2] = 0
    
    np.fill_diagonal(adjacency_matrix,0)
    
    return adjacency_matrix

In [53]:
'''function to update cumulative doses.
Includes current dose inputs and decay of previous dose inputs'''
def cumu_dose(d,D_prev,gamma) :
    I = d.sum(axis=1).reshape(N,1)
#     I = 0
    D = (D_prev - (gamma*D_prev*dt)) + I
    return D

In [54]:
'''function to synchronously update the states of the nodes of the system.
This is the second step in the dynamics on the network'''
def upd_indi_state(D,d_star,indi_state_prev) :
    
    z2 = np.random.uniform(size=(N,1))
    z3 = np.random.uniform(size=(N,1))
    
    indi_state = indi_state_prev
    
    indi_state_mask1 = (D>=d_star) & (indi_state_prev==1)
    indi_state[indi_state_mask1] = 2
    
    indi_state_mask21 = (D<d_star) & (indi_state_prev==2) & (r>=z2) & (rho>=z3)
    indi_state[indi_state_mask21] = 1
    
    indi_state_mask22 = (D<d_star) & (indi_state_prev==2) & (r>=z2) & (rho<z3)
    indi_state[indi_state_mask22] = 3
    
    indi_state_mask23 = (D<d_star) & (indi_state_prev==2) & (r<z2)
    indi_state[indi_state_mask23] = 2
    
    return indi_state

In [55]:
'''function to get identify the infected nodes and the suceptible nodes'''
def states(indi_state) :
    infec_indi = []
    suscep_indi = []
    infec_indi = np.transpose(np.where(indi_state==2))
    suscep_indi = np.transpose(np.where(indi_state==1))
    return infec_indi,suscep_indi

In [56]:
'''function keeping track of the new graphs being formed at each simulation step
This helps calculating few properties like out degree and in degree'''
def networkx_graph(coords,edge_list) :
    G = nx.DiGraph()
    pos = {(i): (coords[i][0],coords[i][1]) for i in range(N)}
    G.add_nodes_from(pos.keys())
    G.add_edges_from(edge_list)
    return G

In [57]:
'''function to get the directed edge list from the asymmetrical adjacency matrix'''
def edge_from_adj_mat(adj_mat_list,q):
    edge_list = []
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(adj_mat_list[q]==1))))
    return edge_list

# The main part of the script

In [58]:
'''declaring the required data frames'''

#infected state time series data frame
#columns indicate the time steps
timeseries_infec_frac = pd.DataFrame()

#time series of in degree as columns and rows as nodes
# timeseries_in_degree = pd.DataFrame() 

#time series of out degree as columns and rows as nodes
timeseries_out_degree = pd.DataFrame()

#time series of strongly connected components as columns
# timeseries_connec_comps = pd.DataFrame()

#time series of mean of out degree as columns
timeseries_out_degree_mean = pd.DataFrame() 

In [59]:
'''parameters sets which the simulation uses to run
p_rew_vals are the values of a and b since birth and death rates are equal in this case.
The p_rew vals are set such that each node can create a maximum of 0.1,1,10 and 100 links per second.
start_vals are the number of initially infected individuals. Note this is the fraction but the actual number itself
dose_quantity_vals indicate the homogeneous dose sizes being trasnferred during interaction'''

vals = round(1/((density)*(np.pi)*(rew_r*rew_r)),3)
p_rew_vals = [round(0.1*vals,3),round(vals,3),round(10*vals,3),round(100*vals,3)]
start_vals = [1,10]
dose_quantity_vals = [0.5,2]

In [60]:
for sim in range(70,70+25) : #this is the number of different realisation of the same parameter set. Here 25 samples are being generated for each parameter set
    
    '''returns coordinates, distance matrix, edge_list and the adjacency matrix'''
    dist_mat,coords = calc_dist(N) #node placement
    edge_list = knn(dist_mat,nn,N) #setting some sort of initial network. Here knn is chos
    bef_netrun_org_adj_mat = np.int64(np.zeros((N,N)))
    for i in range(len(edge_list)):
        bef_netrun_org_adj_mat[edge_list[i][0],edge_list[i][1]] = np.int64(1) #the initial/original adjacency matrix
    df_row = 0
    
    for p_rew_org in p_rew_vals : #rate of rewiring (per unit time (second))
        
        '''---------------------------------------------------------------------------------------------------------------
        temporarily changing the rewiring rates in order to speed up the
        preocess of the network topology reaching steady state'''
        if p_rew_org == round(0.1*vals,3) :
            dt_netrun = 100
        elif p_rew_org == round(vals,3):
            dt_netrun = 10
        else :
            dt_netrun = dt
            
        lamb_da_org = p_rew_org/10
        '''-------------------------------------------------------------------------------------------------------------'''
        
        '''the actual parameters for the coupling dynamics'''
        lamb_da = lamb_da_org*dt 
        p_rew = p_rew_org*dt
        
        '''parameters for the network satbilisation'''
        lamb_da_netrun = lamb_da_org*dt_netrun
        p_rew_netrun = p_rew_org*dt_netrun
        
        for start in start_vals : #rate of dose transfer (per unit time (second))
            for dose_quantity in dose_quantity_vals : #number of nodes to infect initially
                print('(sim,lamb_da,p_rew,start,dose_quantity) = ', (sim,lamb_da/dt,p_rew/dt,start,dose_quantity))

                '''clearing exsiting data frames and re-creating/re-declaring them '''
                timeseries_infec_frac = pd.DataFrame()
#                 timeseries_in_degree = pd.DataFrame()
                timeseries_out_degree = pd.DataFrame()
#                 timeseries_connec_comps = pd.DataFrame()
                timeseries_out_degree_mean = pd.DataFrame()

                q = 0 #to generate new edge_list from the new adjacency matrix 
                t = np.arange(0,T,dt)

                adj_mat_list = []
                adj_mat = copy.deepcopy(bef_netrun_org_adj_mat)
                adj_mat_new = copy.deepcopy(adj_mat)
                adj_mat_list.append(adj_mat_new) #list of arrays which shows the time series of the adjacency matrix
    
                
                '''------------------------------------------------------------------------------------------------------------
                obtaining a steady network topology before starting the dynamics on the network'''
                q = 0 #also for navigating the list of adjacency matrices
                steady_counter = 0
                df_col = 0 #for navigating the data frames
                no_reset = 0 
                while steady_counter < steady_counter_val : #if counter exceeds the user set value, network topolgy steady state has been reached 

                    #coords remain the same. Get new edge_list from the latest adjacency matrix
                    edge_list = edge_from_adj_mat(adj_mat_list,q)
                    G = networkx_graph(coords,edge_list) #networkx graph

                    timeseries_out_degree.loc[:,df_col] = np.asarray([val for (node, val) in G.out_degree()])
                    timeseries_out_degree_mean.loc[0,df_col] = timeseries_out_degree.iloc[:,df_col].mean()
                    if df_col-1 >= 0 :
                        
                        #if new network topology is similar to previous, counter increases
                        if np.abs(timeseries_out_degree_mean.loc[0,df_col] - timeseries_out_degree_mean.loc[0,df_col-1]) <= tol :
                            steady_counter = steady_counter + 1
                        
                        #else start restart the counter from 0
                        else :
                            steady_counter = 0
                            no_reset = no_reset + 1
#                             print('re-setting steady_counter!')
                            #to prevent infintie restarts,asssume latest toplogy to be that of steady state. 
                            #This will be still better than the initial network
                            if no_reset > 100 :
                                break

                    OD_array = np.asarray([val for (node, val) in G.out_degree()])
                    OD_mesh1,OD_mesh2 = np.meshgrid(OD_array,OD_array)    
                    adj_mat = let_it_run(p_rew_netrun,adj_mat,OD_mesh2,lamb_da_netrun,density,rew_r)
                    adj_mat_new = copy.deepcopy(adj_mat)
                    adj_mat_list.append(adj_mat_new)

                    q = q + 1 
                    df_col = df_col + 1
                    
                '''steady network topology obtained, start the dynamics now
                ---------------------------------------------------------------------------------------------------------------'''

                '''re-initialising. This time with the steady state network topology'''                
                print(timeseries_out_degree_mean.loc[0,df_col-1])
                timeseries_out_degree = pd.DataFrame()
                df_col = 0 #for navigating the data frames
                q = 0      #for navigating the list of adjacency matrices
                edge_list = copy.deepcopy(edge_list)
                G = copy.deepcopy(G)
                org_adj_mat = copy.deepcopy(adj_mat_new)

#                 numpy_array = np.zeros((N,len(t)),dtype = np.int64)

                D_array = np.zeros((N,len(t)))
                for i in range(len(edge_list)):
                    org_adj_mat[edge_list[i][0],edge_list[i][1]] = np.int64(1) #ideally, this is now the network topolgy which does not remeber the user bias!

                adj_mat_list = []
                adj_mat = copy.deepcopy(org_adj_mat)
                adj_mat_new = copy.deepcopy(adj_mat)
                adj_mat_list.append(adj_mat_new)


                '''from G, obtain the in degree and the out degree'''
#                 timeseries_in_degree.loc[:,0] = np.asarray([val for (node, val) in G.in_degree()])
                timeseries_out_degree.loc[:,0] = np.asarray([val for (node, val) in G.out_degree()])

                '''strongly and weekely connected components'''
#                 timeseries_connec_comps.loc[0,0] = nx.number_connected_components(G.to_undirected())

                '''choosing initially infected nodes'''
                indi_state = np.random.randint(1,2,size=(N,1))
                infected_nodes = None
                while infected_nodes is None : #infecting 'start' number of network neighbours
                    infected_nodes = InfectNetworkNeighbors(G,np.random.randint(N),start)      
                print(infected_nodes)
                infected_nodes = np.asarray(list(infected_nodes)).reshape(len(infected_nodes),1)
                indi_state[infected_nodes[:,0],0] = 2

                A,B = np.meshgrid(indi_state,indi_state) 
                partner_state = copy.deepcopy(A)
                indi_state_for_rew = copy.deepcopy(B)

    #             numpy_array [:,0] = indi_state[:,0]

                indi_state_prev = copy.deepcopy(indi_state)
                partner_state_prev = copy.deepcopy(partner_state)
                indi_state_for_rew_prev = copy.deepcopy(indi_state_for_rew)

                d = np.zeros((len(adj_mat),len(adj_mat[0])))

                D = np.zeros((N,1))
                D[np.where(indi_state==2)] = D_ini
                D_array[:,0] = D[:,0]
                D_prev = copy.deepcopy(D)

                '''obtaining the infected fraction'''
                infec_frac = np.count_nonzero(indi_state == 2)/N
                timeseries_infec_frac.loc[df_row,0] = infec_frac
                
                '''the main part of the simulation'''
                counter = 0
                df_col = 1
                for t in np.arange(dt,T,dt) :
                    q = q + 1 
                    counter = counter + 1
                    infec_indi = []
                    suscep_indi = []
                    d = np.zeros((len(adj_mat),len(adj_mat[0])))
                    d = dose(adj_mat,p,partner_state_prev,d)

                    OD_array = np.asarray([val for (node, val) in G.out_degree()])
                    OD_mesh1,OD_mesh2 = np.meshgrid(OD_array,OD_array)
                    adj_mat = rew(p_rew,adj_mat,indi_state_for_rew_prev,OD_mesh2,lamb_da,density,rew_r)
                    adj_mat_new = copy.deepcopy(adj_mat)
                    adj_mat_list.append(adj_mat_new)

                    #coords remain the same. Get new edge_list from the latest adjacency matrix
                    edge_list = edge_from_adj_mat(adj_mat_list,q)
                    G = networkx_graph(coords,edge_list) #networkx graph

                    '''from G, obtain the in degree and the out degree'''
#                     timeseries_in_degree.loc[:,df_col] = np.asarray([val for (node, val) in G.in_degree()])
                    timeseries_out_degree.loc[:,df_col] = np.asarray([val for (node, val) in G.out_degree()])

                    '''strongly and weekely connected components'''
#                     timeseries_connec_comps.loc[df_row,df_col] = nx.number_connected_components(G.to_undirected())

                    D = cumu_dose(d,D_prev,gamma)
                    D_array[:,counter] = D[:,0]

                    indi_state = upd_indi_state(D,d_star,indi_state_prev)
                    infec_frac = np.count_nonzero(indi_state == 2)/N
                
                    '''obtaining the infected fraction'''
                    timeseries_infec_frac.loc[df_row,df_col] = infec_frac

    #                 numpy_array[:,counter] = indi_state[:,0]
                    A,B = np.meshgrid(indi_state,indi_state)

                    infec_indi, suscep_indi = states(indi_state)

                    partner_state = copy.deepcopy(A)
                    indi_state_for_rew = copy.deepcopy(B)
                    indi_state_prev = copy.deepcopy(indi_state)
                    partner_state_prev = copy.deepcopy(partner_state)
                    indi_state_for_rew_prev = copy.deepcopy(indi_state_for_rew)

                    D_prev = copy.deepcopy(D)
                    D = np.zeros((N,1))
                    df_col = df_col + 1 

    #             df_row = df_row + 1 #going to the next row of the df to store timeseries of next (p,start)

    
                '''saving data to respective files'''
        
                filename_infec_frac = 'infec_frac.h5'
                filename_in_deg = 'in_deg.h5'
                filename_out_deg = 'out_deg.h5'
                filename_connec_comps = 'connec_comps.h5'
                p_rew_val = '%g'%(p_rew/dt)
                key_val = 'sim_'+str(sim)+'_p_rew_'+str(p_rew_val)+'_start_'+str(start)+'_dq_'+str(dose_quantity)
#                 print(key_val)
                timeseries_infec_frac.to_hdf(filename_infec_frac, key = key_val, mode='a')
#                 timeseries_in_degree.to_hdf(filename_in_deg, key = key_val, mode='a')
                timeseries_out_degree.to_hdf(filename_out_deg, key = key_val, mode='a')
#                 timeseries_connec_comps.to_hdf(filename_connec_comps, key = key_val, mode='a')
                df_row = 0

(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.0005, 0.005, 1, 0.5)
7.0
{65}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.0005, 0.005, 1, 2)
6.86
{66}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.0005, 0.005, 10, 0.5)
7.41
{66, 2, 6, 40, 27, 75, 14, 83, 21, 91}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.0005, 0.005, 10, 2)
7.45
{1, 35, 76, 20, 84, 88, 25, 90, 94, 57}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.005099999999999999, 0.051000000000000004, 1, 0.5)
6.9
{22}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.005099999999999999, 0.051000000000000004, 1, 2)
6.8
{9}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.005099999999999999, 0.051000000000000004, 10, 0.5)
7.49
{64, 65, 33, 5, 13, 48, 93, 25, 61, 30}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.005099999999999999, 0.051000000000000004, 10, 2)
7.76
{66, 36, 6, 27, 42, 83, 21, 56, 91, 63}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.051000000000000004, 0.51, 1, 0.5)
8.78
{44}
(sim,lamb_d

(sim,lamb_da,p_rew,start,dose_quantity) =  (74, 0.51, 5.1, 1, 0.5)
7.23
{14}
(sim,lamb_da,p_rew,start,dose_quantity) =  (74, 0.51, 5.1, 1, 2)
7.18
{6}
(sim,lamb_da,p_rew,start,dose_quantity) =  (74, 0.51, 5.1, 10, 0.5)
7.15
{1, 3, 7, 9, 12, 13, 22, 86, 91, 29}
(sim,lamb_da,p_rew,start,dose_quantity) =  (74, 0.51, 5.1, 10, 2)
7.13
{69, 5, 6, 73, 82, 52, 55, 23, 58, 59}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.0005, 0.005, 1, 0.5)
7.34
{43}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.0005, 0.005, 1, 2)
6.73
{86}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.0005, 0.005, 10, 0.5)
6.48
{2, 98, 74, 75, 76, 15, 49, 50, 86, 31}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.0005, 0.005, 10, 2)
7.19
{2, 35, 98, 39, 42, 79, 49, 86, 27, 62}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.005099999999999999, 0.051000000000000004, 1, 0.5)
6.51
{32}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.005099999999999999, 0.051000000000000004, 1, 2)
6.99
{35}
(sim,lamb_da,p_rew,

8.22
{29}
(sim,lamb_da,p_rew,start,dose_quantity) =  (79, 0.051000000000000004, 0.51, 1, 2)
8.28
{26}
(sim,lamb_da,p_rew,start,dose_quantity) =  (79, 0.051000000000000004, 0.51, 10, 0.5)
8.13
{33, 68, 70, 72, 41, 42, 44, 77, 82, 91}
(sim,lamb_da,p_rew,start,dose_quantity) =  (79, 0.051000000000000004, 0.51, 10, 2)
8.25
{39, 75, 18, 51, 84, 54, 55, 22, 57, 94}
(sim,lamb_da,p_rew,start,dose_quantity) =  (79, 0.51, 5.1, 1, 0.5)
8.16
{87}
(sim,lamb_da,p_rew,start,dose_quantity) =  (79, 0.51, 5.1, 1, 2)
7.58
{79}
(sim,lamb_da,p_rew,start,dose_quantity) =  (79, 0.51, 5.1, 10, 0.5)
8.22
{32, 33, 35, 36, 43, 11, 78, 85, 88, 62}
(sim,lamb_da,p_rew,start,dose_quantity) =  (79, 0.51, 5.1, 10, 2)
8.26
{66, 3, 68, 41, 44, 78, 82, 53, 89, 94}
(sim,lamb_da,p_rew,start,dose_quantity) =  (80, 0.0005, 0.005, 1, 0.5)
7.61
{76}
(sim,lamb_da,p_rew,start,dose_quantity) =  (80, 0.0005, 0.005, 1, 2)
6.83
{46}
(sim,lamb_da,p_rew,start,dose_quantity) =  (80, 0.0005, 0.005, 10, 0.5)
7.18
{66, 7, 75, 12, 14, 53, 

6.45
{48}
(sim,lamb_da,p_rew,start,dose_quantity) =  (84, 0.005099999999999999, 0.051000000000000004, 10, 0.5)
7.17
{0, 3, 6, 40, 75, 45, 79, 22, 60, 94}
(sim,lamb_da,p_rew,start,dose_quantity) =  (84, 0.005099999999999999, 0.051000000000000004, 10, 2)
6.44
{99, 5, 42, 44, 50, 84, 85, 53, 21, 59}
(sim,lamb_da,p_rew,start,dose_quantity) =  (84, 0.051000000000000004, 0.51, 1, 0.5)
7.91
{6}
(sim,lamb_da,p_rew,start,dose_quantity) =  (84, 0.051000000000000004, 0.51, 1, 2)
8.12
{17}
(sim,lamb_da,p_rew,start,dose_quantity) =  (84, 0.051000000000000004, 0.51, 10, 0.5)
7.86
{98, 12, 80, 54, 23, 24, 86, 26, 27, 61}
(sim,lamb_da,p_rew,start,dose_quantity) =  (84, 0.051000000000000004, 0.51, 10, 2)
7.7
{96, 0, 3, 42, 76, 79, 50, 21, 85, 30}
(sim,lamb_da,p_rew,start,dose_quantity) =  (84, 0.51, 5.1, 1, 0.5)
7.51
{26}
(sim,lamb_da,p_rew,start,dose_quantity) =  (84, 0.51, 5.1, 1, 2)
7.42
{4}
(sim,lamb_da,p_rew,start,dose_quantity) =  (84, 0.51, 5.1, 10, 0.5)
6.9
{64, 66, 37, 7, 72, 41, 14, 82, 54, 2

6.42
{1, 66, 68, 38, 48, 50, 52, 26, 91, 30}
(sim,lamb_da,p_rew,start,dose_quantity) =  (89, 0.0005, 0.005, 10, 2)
6.99
{33, 3, 37, 69, 7, 79, 17, 21, 89, 61}
(sim,lamb_da,p_rew,start,dose_quantity) =  (89, 0.005099999999999999, 0.051000000000000004, 1, 0.5)
6.5
{46}
(sim,lamb_da,p_rew,start,dose_quantity) =  (89, 0.005099999999999999, 0.051000000000000004, 1, 2)
6.98
{25}
(sim,lamb_da,p_rew,start,dose_quantity) =  (89, 0.005099999999999999, 0.051000000000000004, 10, 0.5)
6.93
{6, 39, 74, 29, 51, 22, 55, 88, 59, 93}
(sim,lamb_da,p_rew,start,dose_quantity) =  (89, 0.005099999999999999, 0.051000000000000004, 10, 2)
7.22
{96, 34, 41, 44, 14, 53, 30, 56, 58, 62}
(sim,lamb_da,p_rew,start,dose_quantity) =  (89, 0.051000000000000004, 0.51, 1, 0.5)
7.56
{35}
(sim,lamb_da,p_rew,start,dose_quantity) =  (89, 0.051000000000000004, 0.51, 1, 2)
7.86
{36}
(sim,lamb_da,p_rew,start,dose_quantity) =  (89, 0.051000000000000004, 0.51, 10, 0.5)
7.75
{98, 2, 69, 39, 8, 74, 79, 24, 27, 94}
(sim,lamb_da,p_rew

7.35
{33, 98, 8, 77, 45, 47, 80, 20, 94, 63}
(sim,lamb_da,p_rew,start,dose_quantity) =  (93, 0.51, 5.1, 10, 2)
7.41
{99, 4, 39, 73, 46, 50, 19, 85, 53, 90}
(sim,lamb_da,p_rew,start,dose_quantity) =  (94, 0.0005, 0.005, 1, 0.5)
7.54
{49}
(sim,lamb_da,p_rew,start,dose_quantity) =  (94, 0.0005, 0.005, 1, 2)
6.85
{12}
(sim,lamb_da,p_rew,start,dose_quantity) =  (94, 0.0005, 0.005, 10, 0.5)
7.58
{3, 7, 11, 43, 13, 19, 84, 22, 55, 24}
(sim,lamb_da,p_rew,start,dose_quantity) =  (94, 0.0005, 0.005, 10, 2)
6.62
{65, 34, 67, 8, 75, 54, 87, 58, 60, 95}
(sim,lamb_da,p_rew,start,dose_quantity) =  (94, 0.005099999999999999, 0.051000000000000004, 1, 0.5)
6.77
{81}
(sim,lamb_da,p_rew,start,dose_quantity) =  (94, 0.005099999999999999, 0.051000000000000004, 1, 2)
7.36
{6}
(sim,lamb_da,p_rew,start,dose_quantity) =  (94, 0.005099999999999999, 0.051000000000000004, 10, 0.5)
7.4
{33, 35, 5, 43, 13, 17, 19, 84, 90, 61}
(sim,lamb_da,p_rew,start,dose_quantity) =  (94, 0.005099999999999999, 0.051000000000000004,